In [10]:
from datasets import Dataset


data = {
    "question": [
    "痞满是什么疾病的临床症状",
    "治疗痞满应采用什么治疗方法",
    "枳实消痞丸治疗有痞满症状的慢性萎缩性胃炎是什么原理？",
    "使用枳实消痞丸的具体治疗处方可以有什么变通？",
    "介绍一下反流性食道炎的典型类型，分析其病机。",
    "反流性食道炎应该用什么方治疗？",
    "患者自诉最近一段时间逐渐出现胸骨后不适，偶有灼痛，泛酸，呃逆，伴咯痰黏稠，咯之不爽，口苦。可以观察到患者舌红苔黄稍腻，脉滑。",
    "根据前面诊治后，患者胸骨后灼痛较前好转，但食入后反酸，呃逆，胃脘疼痛，咯黏痰，咯之不利，夜间休息差，舌淡红，苔黄少，脉缓。应如何诊治？"
    ],
    "answer": [
    "痞满是慢性萎缩性胃炎的主要临床症状之一，也可称为胃痞。",
    "治疗痞满可采用张仲景半夏泻心汤的核心配伍组药，即半夏配黄连，体现了寒热并用，辛开苦降，消痞散结的配伍组方治疗思想。",
    "枳实消痞丸是在半夏泻心汤、枳术丸的基础上取其核心配伍衍化而来，可治疗脾胃虚弱，寒热互结之痞满。",
    "可以抽取辛开苦降，消痞散结之半夏、黄连、枳实，作为配伍组药之核心，用于治疗慢性萎缩性胃炎寒热互结，脾胃气滞之胃脘痞满。若病发于脾胃气虚，而见胃脘痞满，不思饮食，倦怠乏力，常配香砂六君子汤。",
    "反流性食道炎若胸骨后不适，胃脘痞满，反流，口苦，为痰热互结，胃气逆阻为患；若胸骨后有灼热感，口干不欲饮，为胃阴不足。",
    "痰热互结，胃气逆阻的反流性食道炎，可用张仲景小陷胸汤 (半夏、黄连、瓜蒌)配苏梗形成配伍组药开痰结、降胃气治疗。其中半夏与黄连配伍辛开苦降，消痞散结，瓜蒌清热化痰，宽胸润降，得半夏；黄连清热开结化痰，配苏梗宽胸润降胃气。\n胃阴不足的反流性食道炎药配自拟滋胃汤(太子参、麦冬、石斛)，以润为降，反酸配刺猬皮制酸和胃。",
    "中医辨证为痰热结胸，胃气逆阻的反流性食道炎。治以辛开苦降，清化痰热，润降胃气。",
    "中医证属痰热，气血不畅兼阴虚，治以清 化痰热，滋阴润降，行气活血。"
    ]
}


dataset = Dataset.from_dict(data)


In [11]:
def format_data(samples):
    question = samples['question']
    answer = samples['answer']
    prompt = f"### QUESTION\n{question}\n\n### ANSWER\n{answer}</s>"
    samples['text'] = prompt
    return samples

In [12]:
dataset = dataset.map(
    format_data,
    remove_columns=['question', 'answer']
)

Map: 100%|██████████| 8/8 [00:00<00:00, 1196.24 examples/s]


In [13]:
dataset.save_to_disk("../data/datasets/zyqa")

Saving the dataset (1/1 shards): 100%|██████████| 8/8 [00:00<00:00, 1421.20 examples/s]


In [14]:
dataset

Dataset({
    features: ['text'],
    num_rows: 8
})

In [15]:
dataset[0]

{'text': '### QUESTION\n痞满是什么疾病的临床症状\n\n### ANSWER\n痞满是慢性萎缩性胃炎的主要临床症状之一，也可称为胃痞。</s>'}